In [36]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import datetime
import pickle
from shapely.geometry import Point

In [30]:
promise_zone = gpd.read_file('../data/MDHA_Promise_Zones/Export_Output_5.shp')
print(promise_zone.crs)
promise_zone.head(7 )

epsg:2274


,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe,geometry
0,1,1,1.053728e+08,49364.601403,"POLYGON ((1743366.812 678044.543, 1743367.698 ..."
1,2,2,7.670585e+07,45226.018917,"POLYGON ((1738690.545 661791.347, 1738733.644 ..."
2,3,3,2.985488e+08,75207.067487,"POLYGON ((1759602.544 668309.889, 1759640.155 ..."
3,4,4,2.713638e+08,74348.596054,"POLYGON ((1747799.003 654974.612, 1747821.293 ..."
4,5,5,3.928178e+08,106661.712102,"POLYGON ((1737013.744 692634.639, 1737122.605 ..."
5,6,6,9.535996e+07,59256.923574,"POLYGON ((1716935.654 667534.495, 1716992.481 ..."


In [31]:
promise_zone = promise_zone.to_crs(epsg = 4326)

In [32]:
polygon_zone_6 = promise_zone.loc[promise_zone['ZONE_ID']==6]
polygon_zone_6.head()

,OBJECTID,ZONE_ID,SHAPE_STAr,SHAPE_STLe,geometry
5,6,6,9.535996e+07,59256.923574,"POLYGON ((-86.85223 36.16417, -86.85204 36.164..."


In [33]:
bus_stops = pd.read_csv('../data/busstops_cleaned.csv')
print(bus_stops.shape)
bus_stops.head(3)

(2524, 5)


,stop,route,location,lat,lng
0,GREEN LN & WHITES CREEK PIKE WB,GOLDEN VALLEY,"(36.236249, -86.816722)",36.236249,-86.816722
1,_ 9TH AVE S & EDGEHILL AVE SB,8TH AVENUE SOUTH,"(36.142642, -86.780897)",36.142642,-86.780897
2,DONELSON/DELL STATION OUTBOUND,MURFREESBORO PIKE,"(36.105615, -86.672004)",36.105615,-86.672004


In [37]:
bus_stops['geometry'] = bus_stops.apply(lambda x: Point((x.lng,x.lat)),
                                        axis=1)
bus_stops.head(3)

,stop,route,location,lat,lng,geometry
0,GREEN LN & WHITES CREEK PIKE WB,GOLDEN VALLEY,"(36.236249, -86.816722)",36.236249,-86.816722,POINT (-86.816722 36.236249)
1,_ 9TH AVE S & EDGEHILL AVE SB,8TH AVENUE SOUTH,"(36.142642, -86.780897)",36.142642,-86.780897,POINT (-86.780897 36.142642)
2,DONELSON/DELL STATION OUTBOUND,MURFREESBORO PIKE,"(36.105615, -86.672004)",36.105615,-86.672004,POINT (-86.672004 36.105615)


In [39]:
bus_geo = gpd.GeoDataFrame(bus_stops, 
                           crs = promise_zone.crs, 
                           geometry = bus_stops['geometry'])

In [40]:
stops_by_zone c= gpd.sjoin( bus_geo,promise_zone, op = 'within')

C:\Users\RMAUR\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [41]:
stops_in_zone6 = stops_by_zone.loc[stops_by_zone['ZONE_ID']==6]

In [43]:
stops_by_zone['ZONE_ID'].value_counts()

5    366
3    177
1    127
2    110
4     84
6     50
Name: ZONE_ID, dtype: int64

In [44]:
zone_6_center = polygon_zone_6.centroid
print(zone_6_center)

5    POINT (-86.86377 36.15331)
dtype: geometry


C:\Users\RMAUR\AppData\Local\Temp/ipykernel_2132/2329981329.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zone_6_center = polygon_zone_6.centroid


In [46]:
zone_6_area = (zone_6_center.y,zone_6_center.x)
print(zone_6_area)

(5    36.153311
dtype: float64, 5   -86.863769
dtype: float64)


In [47]:
map_area_6 = folium.Map(location =  zone_6_area, zoom_start = 12)

folium.GeoJson(polygon_zone_6).add_to(map_area_6)

for row_index, row_values in stops_in_zone6.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(map_area_6)

map_area_6

In [2]:
# Import the file as a Pandas DataFrame
fp = '../data/july.csv'
july = pd.read_csv(fp)

In [3]:
company_dict = {'Bird':0, 'Lyft': 1, 'Gotcha': 2, 'Lime': 3, 'Spin': 4, 'Jump': 5, 'Bolt': 6}
july.companyname = july.companyname.replace(company_dict)
july.head()

,pubdatetime,latitude,longitude,sumdid,sumdtype,chargelevel,sumdgroup,costpermin,companyname
0,2019-07-01 00:00:33.550000,36.156678,-86.809004,Powered635135,Powered,22.0,scooter,0.15,1
1,2019-07-01 00:00:34.973000,36.145674,-86.794138,Powered790946,Powered,33.0,scooter,0.15,1
2,2019-07-01 00:00:41.183000,36.179319,-86.751538,Powered570380,Powered,76.0,scooter,0.15,1
3,2019-07-01 00:00:41.620000,36.152111,-86.803821,Powered240631,Powered,43.0,scooter,0.15,1
4,2019-07-01 00:00:45.087000,36.149355,-86.797550,Powered970404,Powered,52.0,scooter,0.15,1


In [15]:
july.pubdatetime = pd.to_datetime(july['pubdatetime'])

In [23]:
july.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25075445 entries, 0 to 25075444
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   pubdatetime  datetime64[ns]
 1   latitude     float64       
 2   longitude    float64       
 3   sumdid       object        
 4   sumdtype     object        
 5   chargelevel  float64       
 6   sumdgroup    object        
 7   costpermin   float64       
 8   companyname  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 1.7+ GB


In [5]:
july.to_pickle("../data/july.pkl")

In [6]:
july_scooters = july[['pubdatetime', 'latitude', 'longitude', 'sumdid', 'chargelevel', 'companyname']]

In [7]:
##july 1 was a monday
july_1_scooters = july_scooters[july_scooters['pubdatetime']<='2019-07-02 00:00:00.000'] 

In [21]:
july_1_scooters['pubdatetime'] = pd.to_datetime(july_1_scooters['pubdatetime'])

C:\Users\RMAUR\AppData\Local\Temp/ipykernel_2132/1990105203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july_1_scooters['pubdatetime'] = pd.to_datetime(july_1_scooters['pubdatetime'])


In [28]:
july_1_scooters = july_1_scooters.set_index('pubdatetime')

In [54]:
july_1_scooters['geometry'] = july_1_scooters.apply(lambda x: Point(float(x.longitude), float(x.latitude)),axis=1)

In [62]:
scoots_geo = gpd.GeoDataFrame(july_1_scooters,crs=promise_zone.crs,geometry=july_1_scooters.geometry)

In [64]:
scoots_by_zone = gpd.sjoin( scoots_geo,promise_zone, op = 'within')

In [65]:
jul1_scoots_zone6 = scoots_by_zone.loc[scoots_by_zone['ZONE_ID']==6]

In [66]:
# Create an empty list
df_hour_list = []

# Create a series with the different hours of the day
hours = pd.Series(jul1_scoots_zone6.index.hour.unique().sort_values())

# Create a list of points for each hour of the day
def create_list(hour):
    df_hour_list.append(jul1_scoots_zone6.loc[jul1_scoots_zone6.index.hour == hour,
                                ['latitude',
                                 'longitude']].
                        groupby(['latitude', 
                                 'longitude']).sum().reset_index().values.tolist())
hours.apply(create_list);

In [69]:
from folium.plugins import HeatMapWithTime

# Add trip events to the map
scoot_map_time = folium.Map(location = zone_6_area, 
                      tiles="CartoDB Positron", 
                      zoom_start=12)


folium.GeoJson(polygon_zone_6).add_to(scoot_map_time)

for row_index, row_values in stops_in_zone6.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(scoot_map_time)

HeatMapWithTime(df_hour_list, 
                auto_play=False, 
                max_opacity=0.5, 
                gradient = {0.2: '#FBD973', 
                            0.4: '#fa782f', 
                            0.75: '#F16578', 
                            1: '#782890'}).add_to(scoot_map_time)
scoot_map_time.save('../maps/scoot_map_time.html')
scoot_map_time